In [1]:
%matplotlib inline
import os, io, json, urllib, numpy as np, codecs
import glob, pandas as pd
pd.options.display.max_rows = 999

In [24]:
def h_(citation):
    citation=list(citation)
    h=0
    citation.sort(reverse=True)
    for i in range(1,len(citation)+1):
        if citation[i-1] >= i:
            h=i
    return h

In [10]:
# skip if result.txt exists
y16 = glob.glob('data/2016/*.txt')
y15 = glob.glob('data/2015/*.txt')
y14 = glob.glob('data/2014/*.txt')
y14_16=y14+y15+y16
with open("data/data.txt", "wb") as outfile:
    for f in y14_16:
        with open(f, "rb") as infile:
            lines=infile.read().split('\n')
            for line in lines:
                if len(line.split('\t'))==9:
                    outfile.write(line+'\n')

In [137]:
# skip if result.txt exists
y14 = glob.glob('data/2014'+'/*.txt')
with open("data/2014.txt", "wb") as outfile:
    for f in y14:
        with open(f, "rb") as infile:
            lines=infile.read().split('\n')
            for line in lines:
                if len(line.split('\t'))==10:
                    outfile.write(line+'\n')

In [138]:
df=pd.read_table('data/2014.txt',header=None)
df.columns=['doi','year','ref','citation','issnp','issne','journal','publisher','author','lic']
df.shape

(889986, 10)

In [139]:
#df=df[df['author']==1]
#df.shape

In [140]:
df['year']=df['year'].astype(int)

In [141]:
df=df[df['year']<=2016][df['year']>=2014]
df.shape

C:\Users\moqri\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


(882444, 10)

In [142]:
df['count']=1
df['cited']=df['citation'].apply(lambda x: 1 if x>0 else 0)
df['un-cited']=df['citation'].apply(lambda x: 0 if x>0 else 1)

In [143]:
df=df[(~df.issnp.isnull()) | (~df.issne.isnull())]
df.shape

(882412, 13)

In [144]:
df=df[['year','count','citation','issnp','issne']].fillna('_')
df['issn']=df.issnp+','+df.issne
df['issn']=df.issn.str.replace(',_','')
df['issn']=df.issn.str.replace('_,','')

In [145]:
df['issn']=df.issn.apply(lambda x:x[0:9] if x[10:19]==x[0:9] else x)

In [146]:
s=df.issn.str.split(',',expand=True).stack()

In [147]:
i = s.index.get_level_values(0)

In [148]:
df2 = df.loc[i].copy()

In [149]:
df2["@id"] = s.values

In [166]:
dg=df2.groupby(['@id','year']).agg({'count':sum,'citation':sum}).reset_index()
dg['impact']=dg['citation']/dg['count']
dg.shape

(37176, 5)

In [204]:
dg_h=df2[['@id','citation']].groupby(['@id']).agg(lambda x:tuple(x)).reset_index()
dg_h['h']=dg2.citation.apply(lambda x: h_(x))
dg_h.shape

(37176, 3)

In [169]:
dg_i=df2.groupby(['@id']).agg({'count':sum,'citation':sum}).reset_index()
dg_i['impact']=dg_i['citation']/dg_i['count']
dg_i.shape

(37176, 4)

In [223]:
dg_flat=dg.pivot(index='@id',columns='year')
dg_flat.columns = [col[0]+'_'+str(col[1])[3] for col in dg_flat.columns.values]
dg_flat=dg_flat.reset_index()

In [228]:
dg_merge=dg_flat.merge(dg_i[['@id','impact']]).merge(dg_h[['@id','h']])
dg_merge.shape

(37176, 6)

In [229]:
dic=[]
for index, i in dg_merge.iterrows():
    journal={}
    journal['@id']=i['@id']
    #issn_list=[i['@id']]
    #if (i['issne2']!='') and (i['issne2']!=i['@id']):
        #issn_list.append(i['issne2'])
    #journal['e:issn']=issn_list
    journal['e:count_4']=i['count_4']
    journal['e:citation_4']=i['citation_4']
    journal['e:count_5']=i['count_5']
    journal['e:citation_5']=i['citation_5']
    journal['e:count_6']=i['count_6']
    journal['e:citation_6']=i['citation_6']
    journal['e:count']=i['count']
    journal['e:citation']=i['citation']
    journal['e:h']=i['h']
    journal['e:impact']=round(i['impact'],2)
    if journal not in dic:
        dic.append(journal)

KeyError: 'count_5'

In [183]:
with open('journals_stats.jsonld', 'w') as outfile:
    json.dump(dic, outfile)